### Import Python packages

In [ ]:
import numpy as np
import pandas as pd
import warnings
from time import time
warnings.filterwarnings('ignore')
from scipy import signal
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt
import math
import pickle
import statistics
from sklearn.metrics.pairwise import cosine_similarity
from math import log
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import accuracy_score,recall_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
# from xgboost import XGBClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
from collections import Counter

### Generate EE vector names

In [ ]:
#Input comes from calculating the length of the EE vector generated for this case
a = 100
b = 50
c = 34
d = 25
e = 20
f = 17
g = 15
h = 13
i = 12
j = 10

# Create a list comprehension to generate the range of numbers with the prefix
numbers_1 = [f"0.5_{i}" for i in range(1, a)]
numbers_2 = [f"1.0_{i}" for i in range(1, b)]
numbers_3 = [f"1.5_{i}" for i in range(1, c)]
numbers_4 = [f"2.0_{i}" for i in range(1, d)]
numbers_5 = [f"2.5_{i}" for i in range(1, e)]
numbers_6 = [f"3.0_{i}" for i in range(1, f)]
numbers_7 = [f"3.5_{i}" for i in range(1, g)]
numbers_8 = [f"4.0_{i}" for i in range(1, h)]
numbers_9 = [f"4.5_{i}" for i in range(1, i)]
numbers_10 = [f"5.0_{i}" for i in range(1, j)]

merged_list1 = numbers_1 + numbers_2 + numbers_3 + numbers_4 + numbers_5 + numbers_6 + numbers_7 + numbers_8 + numbers_9 + numbers_10
# display(merged_list1)

### Import files

In [ ]:
path_train = f'/FM/EE_values_train_slope.csv'
EE_values_train_slope =  pd.read_csv(path_train)
path_test = f'/FM/EE_values_test_slope.csv'
EE_values_test_slope =  pd.read_csv(path_test)

### Form dataframes

In [ ]:
df_mod = (pd.concat([EE_values_train_slope.iloc[:,1:],EE_values_test_slope.iloc[:,1:]],axis = 0).reset_index()).drop(columns='index')

In [ ]:
X_mod = df_mod.iloc[:,:-1].values
y_mod = df_mod.iloc[:, -1].values

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X_mod)
# display(X)

In [ ]:
# Create a dictionary that maps old column names to new ones
col_dict = {old_col: new_col for old_col, new_col in zip((pd.DataFrame(X)).columns, merged_list1)}

# Rename the columns using the rename() method
df = (pd.DataFrame(X)).rename(columns=col_dict)

# Print the updated dataframe
# display(df)

In [ ]:
X = df
display(X.head())

### Label Encode the data

In [ ]:
enc=LabelEncoder()
if type(y_mod[0])==str:
    y = y_mod.reshape(-1, 1)
    y = pd.DataFrame(enc.fit_transform(y))
else:
    y = y_mod

In [ ]:
display(y.head())

### Feature Selection

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

# min_features_to_select = 1  # Minimum number of features to consider
clf = RandomForestClassifier(random_state=42)
cv = StratifiedKFold(5)
rfecv = RFECV(estimator=clf, step=10, cv=cv, scoring="recall", min_features_to_select= 20, n_jobs=-1)
rfecv.fit(X, y)

display(f"Optimal number of features: {rfecv.n_features_}")

In [ ]:
# get the boolean mask of selected features
feature_mask = rfecv.support_

# select the relevant features from the original dataset
selected_features = X.columns[feature_mask]

# print the selected features
display(selected_features)

In [ ]:
X_selected = rfecv.transform(X)
y_selected = y

In [ ]:
X = X_selected
display((pd.DataFrame(X)).head())
y = y
display((pd.DataFrame(y)).head())

In [ ]:
pickle_file_path = 'x_values.pkl'
(pd.DataFrame(X)).to_pickle(pickle_file_path)
display(f"DataFrame saved as Pickle at {pickle_file_path}")

In [ ]:
pickle_file_path_1 = 'y_values.pkl'
(pd.DataFrame(y)).to_pickle(pickle_file_path_1)
display(f"DataFrame saved as Pickle at {pickle_file_path_1}")